## 1 Importar liberias

In [6]:
import bs4 as bs 
import requests
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
import html5lib as html5lib

## 2 Carga de Tablas 

FORMA 1

In [15]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks')
soup = bs.BeautifulSoup(response.text, "lxml")
Tabla = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in Tabla.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker) #ty

In [ ]:
tickers = [s.replace('\n', '') for s in tickers]
start = datetime.datetime(2000,1,1)
end = datetime.datetime(2021,12,31)
df_list = list()
for ticker in tickers:
    data = yf.download(ticker, group_by="Ticker", start=start, end=end)
    data['ticker'] = ticker  # add this column because the dataframe doesn't contain a column with the ticker
    df_list.append(data)
    sp500 = pd.concat(df_list)

In [17]:
sp500.to_csv('sp500.csv', decimal=',',sep=';')


In [18]:
SP500 = pd.read_csv('sp500.csv', decimal=',',sep=';')

FORMA 2

In [19]:
SPWiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks')
SPWiki = SPWiki[0]

In [20]:
SPWiki.to_csv('SPWiki.csv')

In [21]:
SPWiki = pd.read_csv('SPWiki.csv')

In [22]:
SPWiki.head(5)

,Unnamed: 0,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


## 3 Orden de Tablas

In [23]:
SPWiki.rename(columns={'Symbol':'ticker'}, inplace=True)

In [24]:
SPWiki.drop(columns=['Unnamed: 0', 'Security', 'SEC filings','Headquarters Location', 'Date first added', 'CIK', 'Founded'], inplace=True)

In [25]:
SP500.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,ticker
0,2000-01-03 00:00:00-05:00,48.03125,48.25000,47.03125,47.1875,26.166328,2173400.0,MMM
1,2000-01-04 00:00:00-05:00,46.43750,47.40625,45.31250,45.3125,25.126608,2713800.0,MMM
2,2000-01-05 00:00:00-05:00,45.56250,48.12500,45.56250,46.6250,25.854406,3699400.0,MMM
3,2000-01-06 00:00:00-05:00,47.15625,51.25000,47.15625,50.3750,27.933851,5975800.0,MMM
4,2000-01-07 00:00:00-05:00,50.56250,51.90625,49.96875,51.3750,28.488367,4101200.0,MMM


In [26]:
SP500 = SP500[['ticker','Date','Open','High','Low','Close','Adj Close', 'Volume']]

In [27]:
SP500.head(5)

,ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,MMM,2000-01-03 00:00:00-05:00,48.03125,48.25000,47.03125,47.1875,26.166328,2173400.0
1,MMM,2000-01-04 00:00:00-05:00,46.43750,47.40625,45.31250,45.3125,25.126608,2713800.0
2,MMM,2000-01-05 00:00:00-05:00,45.56250,48.12500,45.56250,46.6250,25.854406,3699400.0
3,MMM,2000-01-06 00:00:00-05:00,47.15625,51.25000,47.15625,50.3750,27.933851,5975800.0
4,MMM,2000-01-07 00:00:00-05:00,50.56250,51.90625,49.96875,51.3750,28.488367,4101200.0


In [ ]:
fecha = pd.to_datetime(SP500['Date'])
SP500['Dia'] = fecha.dt.day_name()
SP500['Mes'] = fecha.dt.month
SP500['Año'] = fecha.dt.year

## 4 Calculos

1 Cual es el mejor día para invertir teniendo en cuenta el retorno de los movimiento gap?

In [29]:
SP500['Gaps'] = round((np.log(SP500['Open']/SP500['Close'].shift(1)).fillna(0)),4)
SP500['Intradiarios'] = round((np.log(SP500['Close']/SP500['Open']).fillna(0)),4)
SP500['Variaciones'] = round((SP500['Adj Close'].pct_change()),4)
SP500['Volatilidad'] = round((SP500.Variaciones.rolling(250,min_periods=1).std()*100*(250)**0.5),4)

In [ ]:
SP500 = SP500[['ticker','Date','Año','Mes','Dia','Open','High','Low','Close','Adj Close', 'Volume','Gaps','Variaciones','Volatilidad','Intradiarios']]

In [31]:
SP500.fillna(0, inplace=True)

In [ ]:
Lunes = SP500['Dia'] == 'Monday'
dflunes = SP500[Lunes]
dflunes.head(5)

In [ ]:
dflunes['Gaps'].sum()

In [ ]:
Martes = SP500['Dia'] == 'Tuesday'
dfmartes = SP500[Martes]

Miercoles = SP500['Dia'] == 'Wednesday'
dfmiercoles = SP500[Miercoles]

Jueves = SP500['Dia'] == 'Thursday'
dfjueves = SP500[Jueves]

Viernes = SP500['Dia'] == 'Friday'
dfviernes = SP500[Viernes]

print(dfmartes['Gaps'].sum(),
dfmiercoles['Gaps'].sum(),
dfjueves['Gaps'].sum(),
dfviernes['Gaps'].sum())

RESPUESTA: Lunes.

2 Cual es el mejor día para invertir teniendo en cuenta el retorno de los movimientos intradiarios?

In [ ]:
print(dflunes['Intradiarios'].sum(), dfmartes['Intradiarios'].sum(),
dfmiercoles['Intradiarios'].sum(),
dfjueves['Intradiarios'].sum(),
dfviernes['Intradiarios'].sum())

RESPUESTA: Jueves.

3 Cuales son las mejores industrias que pertenecen al SP500 en las cuales se puede invertir?

In [ ]:
dfSector = SP500.merge(SPWiki, how='inner', on='ticker')
dfpbi = dfSector['GICS Sector'].to_csv('dfpbi.csv')

In [ ]:
ind = dfSector['GICS Sector'] == 'Industrials'
dfInd = dfSector[ind] 
a2021 = dfInd['Año'] == 2021
dfInd2021 = dfInd[a2021]
dfInd2021.head(5)


In [ ]:
dfInd2021.sort_values('Variaciones', inplace=True, ascending=False)
dfInd2021.head(10)

In [ ]:
EFX = dfInd2021['ticker'] == 'EFX'  
dfEFX = dfInd2021[EFX]

GNRC = dfInd2021['ticker'] == 'GNRC'  
dfGNRC = dfInd2021[GNRC]

PWR = dfInd2021['ticker'] == 'PWR'  
dfPWR = dfInd2021[PWR]

AOS = dfInd2021['ticker'] == 'AOS'  
dfAOS = dfInd2021[AOS]

PCAR = dfInd2021['ticker'] == 'PCAR'  
dfPCAR = dfInd2021[PCAR]

print(dfEFX['Variaciones'].sum(),
      dfGNRC['Variaciones'].sum(),
      dfPWR['Variaciones'].sum(),
      dfAOS['Variaciones'].sum(),
      dfPCAR['Variaciones'].sum())

In [ ]:
print(dfEFX['Volatilidad'].sum(),
      dfGNRC['Volatilidad'].sum(),
      dfPWR['Volatilidad'].sum(),
      dfAOS['Volatilidad'].sum(),
      dfPCAR['Volatilidad'].sum())

In [ ]:
print(dfEFX['Gaps'].sum(),
      dfGNRC['Gaps'].sum(),
      dfPWR['Gaps'].sum(),
      dfAOS['Gaps'].sum(),
      dfPCAR['Gaps'].sum())

RESPUESTA: se encuentra en power bi

4 Cuales fueron los momentos de alta volatilidad que afectaron al SP500?

In [ ]:
SP500.tail(5)

In [ ]:
dfvol = SP500.to_csv('dfvol.csv',sep=';', decimal=',')

RESPUESTA: se encuentra en power bi

5 Cuales son las 9 mejores empresas para invertir?

In [ ]:
a2021 = SP500['Año'] == 2021
df2021 = SP500[a2021]
df2021.sort_values('Variaciones', inplace=True, ascending=False)
df2021.head(10)

In [ ]:
#4
#segunda mitad del 2000

In [ ]:
a2021 = SP500['Año'] == 2021
df2021 = SP500[a2021]
df2021.sort_values('Variaciones', inplace=True, ascending=False)
df2021.head(10)

In [ ]:
BIIB = df2021['ticker'] == 'BIIB'  
dfBIIB = df2021[BIIB]
dfBIIB.head(5)

In [ ]:
dfBIIB['Variaciones'].sum()

In [ ]:
LUMN = df2021['ticker'] == 'LUMN'  
dfLUMN = df2021[LUMN]

ENPH = df2021['ticker'] == 'ENPH'  
dfENPH = df2021[ENPH]

MRNA = df2021['ticker'] == 'MRNA'  
dfMRNA = df2021[MRNA]

ANET = df2021['ticker'] == 'ANET'  
dfANET = df2021[ANET]

TSLA = df2021['ticker'] == 'TSLA'  
dfTSLA = df2021[TSLA]

HIG = df2021['ticker'] == 'HIG'  
dfHIG = df2021[HIG]

EPAM = df2021['ticker'] == 'EPAM'  
dfEPAM = df2021[EPAM]

NFLX = df2021['ticker'] == 'NFLX'  
dfNFLX = df2021[NFLX]

In [ ]:
print(dfBIIB['Variaciones'].sum(),
      dfLUMN['Variaciones'].sum(),
      dfENPH['Variaciones'].sum(),
      dfMRNA['Variaciones'].sum(),
      dfANET['Variaciones'].sum(),
      dfTSLA['Variaciones'].sum(),
      dfHIG['Variaciones'].sum(),
      dfEPAM['Variaciones'].sum(),
      dfNFLX['Variaciones'].sum())

RESPUESTA: Se encuentra en power bi